<font size=6>**Rendu 1 : moteur de recherche**</font>

Julien Velcin, Université Lyon 2 - Master Humanités Numériques

**Le travail attendu :**

- choisir un corpus assez long (au moins 1000 documents et 20k tokens en tout)
- découpez le corpus en textes courts (par ex. des phrases ou des paragraphes)
- prétraitez-le de différentes manières :
    * TF et TFxIDF
    * taille du vocabulaire (par ex. 500 mots les plus fréquents vs. 2000 mots)
    * avec/sans les mots-outils
- déployer un moteur de recherche sur votre corpus

Il convient bien sûr d'interpréter cet énoncé avec ses propres mots, et peut-être faire des choix (ex. faire tel type de comparaison et pas tel autre). Il s'agit ici d'éléments de correction, donc je ne vais pas plus loin.

Ce qui suit est un squelette de document donné à titre indicatif.

Plan (indicatif) :

- librairies utilisées, fonctions utiles
- acquisition des données
- construction des modèles
- moteur de recherche
- discussion
- conclusion

### librairies utilisées

In [ ]:
import os
import math
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import pandas as pd

### fonctions utiles

In [ ]:
# des options permettent de limiter (ou non) le nombre de lignes/colonnes affichées
# par exemple :
# pd.set_option('display.max_rows', None)

# fonction pour afficher les mots (features) d'un ensemble de lignes
def top_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids if row[i]>0]
    df = pd.DataFrame(top_feats)
    if len(top_feats) > 0:
        df.columns = ['feature', 'score']
    return df

# fonction pour afficher les mots (*features*) d'une ligne en particulier (*row_id*)
# *top_n* indique le nombre de mots maximum à afficher
def top_feats_in_doc(Xtr, features, row_id, top_n=25):
    ''' Top features in specific document (matrix row) '''
    row = np.squeeze(Xtr[row_id].toarray())
    return top_feats(row, features, top_n)

# affiche un concordancier pour le motif/mot *pat* dans la chaîne de caractères *texte*
# *window* précise le nombre de caractères à afficher à droite et à gauche
def concord(texte, pat, window = 50):
    pattern = re.compile(pat)
    res = pattern.finditer(texte)
    pos_pattern = [m.span() for m in res]
    context_left = pd.DataFrame({"contexte gauche": [texte[i-window:i-1] for (i, j) in pos_pattern]})
    center = pd.DataFrame({"passage": [texte[i: j] for (i, j) in pos_pattern]})
    context_right = pd.DataFrame({"contexte droit": [texte[j+1:j+window] for (i, j) in pos_pattern]})
    return (pd.concat([context_left, center, context_right], axis=1))

# retourne la similarité du cosinus entre les vecteurs *i* et *j*
def cosinus(i, j):
    num = np.dot(i, j)
    den = math.sqrt(sum(i*i))*math.sqrt(sum(j*j))
    if (den>0):    
        return (num/den)
    else:
        return 0
    
# retourne les *nb_top_docs* documents les plus similaires à la requête *query* dans la matrice *D*
# *features* est associé au vocabulaire construit par la librairie
def search(query, D, features, nb_top_docs=10):
    indexes = [np.where(features == q)[0][0] for q in query if q in features]
    query_vec = np.zeros(len(features))
    query_vec[indexes] = 1
    cc = {i: cosinus(D[i, :], query_vec) for i in range(n_docs)}
    cc = sorted(cc.items(), key=lambda x: x[1], reverse=True)
    return cc[0:nb_top_docs]    

# affiche les *nb_top_docs* premiers documents
def print_docs(result, nb_top_docs=10):
    top_docs = [r for (r,v) in result[0:nb_top_docs]]
    for i, td in zip(range(nb_top_docs), top_docs):
        print("%s (%s): %s" % (i+1, td, docs[td]))

### acquisition des données

(description, motivation et chargement du corpus)

In [ ]:
with open(votre fichier) as f:
    lines = [line.strip() for line in f.readlines()]
    
toute_la_chaine = " ".join(lines).lower()    

In [ ]:
print(f"Nombre de lignes/documents : {len(lines)}")

une_chaine = " ".join(lines)
nb_tokens = len(une_chaine.split(" "))
print(f"Nombre de mots (token) : {nb_tokens}")

Taille du corpus et quelques exemples de textes :

In [ ]:
print("Taille du corpus : {} documents".format(len(lines)))
print("Quelques exemples de textes situés au début du fichier :")
[l for l in lines[0:10]]

In [ ]:
# la variable qui contient les documents
docs = lines

In [ ]:
# le code suivant permet de regrouper les lignes par paragraphes (s'ils sont séparés par des espaces)

docs = []
s = ""
for l in lines:
    if (l != ""):
        s = s + " " + l
    else:
        if (s != ""):
            docs = docs + [s]
            s = ""

In [ ]:
print(f"Nombre de documents : {len(docs)}")

A la fin, il faut que votre ayez constitué votre ensemble de documents dans la liste *docs*

### Construction des modèles

Construction de la matrice Documents x Termes pour différentes configurations

In [ ]:
# ce qui suit est un exemple de configuration possible (TF...)

tf_vectorizer = CountVectorizer()   # ici, on utilise l'encodage TF (mais d'autres sont possibles voire conseillés)
tf_vectorizer.fit(lines)

X = tf_vectorizer.transform(docs)

features = tf_vectorizer.get_feature_names_out() # features permet de sauvegarder les noms des descriptions, càd les mots

In [ ]:
D = X.toarray()
n_docs, n_terms = D.shape

In [ ]:
tf_sum = np.sum(D, axis=0)
top_feats(tf_sum, features, top_n=50)

On peut utiliser le concordancier pour mieux comprendre le corpus (et peut-être enrichir la liste des mots outils).

In [ ]:
concord(toute_la_chaine, mots / suite de mots à rechercher)

Dans ce travail, on vous demander de tester différents paramètres pour le modèle :

- pondération : TF, TFxIDF
- mots-outil : avec ou sans stopwords
- taille : nombre de mots dans le vocabulaire

### moteur de recherche

Cette partie contient les différentes expérimentations que vous avez réalisé avec votre moteur de recherche.
Il faut faire varier le modèle utilisée (sa configuration) mais également les requêtes.
Pensez par exemple à des requêtes utilisant des mots très fréquents et d'autres utilisant des mots rares. Faites aussi varier la taille de la requête (nombre de mots)

In [ ]:
requete = # à compléter
resultat = search(requete, D, features) # où D est la matrice des données et features la liste des mots du vocabulaire
print_docs(resultat)

### discussion

Cette partie doit comporter une discussion sur les résultats obtenus dans la section précédente. Il faut essayer d'être synthétique en tirant quelques conclusions, discuter des points forts et des points faibles.

### conclusion